# Computational Theory Assessment — SHA-256 (FIPS 180-4)

**Student:** Adam Gallagher 

**Student ID:** G00413950

This notebook implements core components of SHA-256 directly from the Secure Hash Standard (FIPS 180-4), using NumPy `uint32` to preserve 32-bit word semantics (wraparound arithmetic and bitwise behavior).


In [ ]:
%pip install numpy

In [8]:
# Imports
import numpy as np
import math
import hashlib

np.set_printoptions(formatter={"int": lambda x: f"{x:#010x}"})


# Problem 1: Binary Words and Operations

**What the problem is**  
- Implement SHA-256 boolean functions and Σ/σ functions using NumPy `uint32` so all operations behave as 32-bit unsigned arithmetic. Document and test each function.

**How I’m going to solve it**  
- Use `np.uint32` for all inputs/outputs.  
- Implement rotate-right (`ROTR`) and logical right shift (`SHR`).  
- Implement `Parity`, `Ch`, `Maj`, `Σ0`, `Σ1`, `σ0`, `σ1` exactly as defined in FIPS 180-4.  
- Add docstrings and minimal correctness tests.


In [15]:
# ---- 32-bit utilities ----

def u32(x) -> np.uint32:
    """Cast to 32-bit unsigned integer."""
    return np.uint32(x)

def rotr(x: np.uint32, n: int) -> np.uint32:
    """
    Rotate-right for 32-bit words.
    ROTR^n(x) = (x >> n) OR (x << (32-n)) with 32-bit wraparound.
    """
    x = u32(x)
    return u32((x >> n) | (x << (32 - n)))

def shr(x: np.uint32, n: int) -> np.uint32:
    """
    Logical right shift for 32-bit words.
    For uint32, right shift is logical (zeros shifted in).
    """
    return u32(u32(x) >> n)

# ---- Problem 1 required functions (FIPS 180-4) ----

def Parity(x: np.uint32, y: np.uint32, z: np.uint32) -> np.uint32:
    """
    Parity(x,y,z) = x XOR y XOR z.
    """
    return u32(u32(x) ^ u32(y) ^ u32(z))

def Ch(x: np.uint32, y: np.uint32, z: np.uint32) -> np.uint32:
    """
    Choice function:
    Ch(x,y,z) = (x AND y) XOR ((NOT x) AND z).
    Bitwise: chooses y when x=1, else chooses z.
    """
    return u32((u32(x) & u32(y)) ^ (~u32(x) & u32(z)))

def Maj(x: np.uint32, y: np.uint32, z: np.uint32) -> np.uint32:
    """
    Majority function:
    Maj(x,y,z) = (x AND y) XOR (x AND z) XOR (y AND z).
    Bitwise majority vote across x,y,z.
    """
    return u32((u32(x) & u32(y)) ^ (u32(x) & u32(z)) ^ (u32(y) & u32(z)))

def Sigma0(x: np.uint32) -> np.uint32:
    """
    Σ0(x) = ROTR^2(x) XOR ROTR^13(x) XOR ROTR^22(x).
    """
    return u32(rotr(x, 2) ^ rotr(x, 13) ^ rotr(x, 22))

def Sigma1(x: np.uint32) -> np.uint32:
    """
    Σ1(x) = ROTR^6(x) XOR ROTR^11(x) XOR ROTR^25(x).
    """
    return u32(rotr(x, 6) ^ rotr(x, 11) ^ rotr(x, 25))

def sigma0(x: np.uint32) -> np.uint32:
    """
    σ0(x) = ROTR^7(x) XOR ROTR^18(x) XOR SHR^3(x).
    """
    return u32(rotr(x, 7) ^ rotr(x, 18) ^ shr(x, 3))

def sigma1(x: np.uint32) -> np.uint32:
    """
    σ1(x) = ROTR^17(x) XOR ROTR^19(x) XOR SHR^10(x).
    """
    return u32(rotr(x, 17) ^ rotr(x, 19) ^ shr(x, 10))

In [21]:
# Problem 1: minimal correctness tests (bitwise identities and rotation/shift sanity)

x, y, z = u32(0x0F0F0F0F), u32(0x33333333), u32(0xAAAAAAAA)

assert Parity(x, y, z) == u32(x ^ y ^ z)
assert Ch(x, y, z) == u32((x & y) ^ (~x & z))
assert Maj(x, y, z) == u32((x & y) ^ (x & z) ^ (y & z))

# Rotation/shift sanity checks
assert rotr(u32(0x80000000), 1) == u32(0x40000000)
assert rotr(u32(0x00000001), 1) == u32(0x80000000)
assert shr(u32(0x80000000), 1) == u32(0x40000000)

print("Problem 1 tests passed.")

Problem 1 tests passed.


## **Problem 2: _Fractional Parts of Cube Roots_**

_<'What does the code do'>_

In [22]:
# Use numpy to calculate the constants listed at the bottom of page 11 of the Secure Hash Standard, following the steps below. 

# These are the first 32 bits of the fractional parts of the cube roots of the first 64 prime numbers.
    #Write a function called primes(n) that generates the first n prime numbers.    

    #Use the function to calculate the cube root of the first 64 primes.

    #For each cube root, extract the first thirty-two bits of the fractional part.

    #Display the result in hexadecimal.

    #Test the results against what is in the Secure Hash Standard.

## **Problem 3: _Padding_**

_<'What does the code do'>_

In [23]:
# Write a generator function block_parse(msg) that processes messages according to section 5.1.1 and 5.2.1 of the Secure Hash Standard. 
# The function should accept a bytes object called msg. At each iteration, it should yield the next 512-bit block of msg as a bytes object. 

# Ensure that the final block (or final two blocks) include the required padding of msg as specified in the standard. 
# Test the generator with messages of different lengths to confirm proper padding and block output.

## **Problem 4: _Hashes_**

_<'What does the code do'>_

In [24]:
# Write a function hash(current, block) that calculates the next hash value given the current hash value and the next message block according to section 6.2.2 SHA-256 Hash Computation on page 22 of the Secure Hash Standard.

## **Problem 5: _Passwords_**

_<'What does the code do'>_

In [25]:
# The following are the SHA-256 hashes of three common passwords that have been hashed using one pass of the SHA-256 algorithm. As strings, they were encoded using UTF-8. 
# Determine the passwords and explain how you found them. Suggest ways in which the hashing of passwords could be improved to prevent the kind of attack you performed to find the passwords.

    # 5e884898da28047151d0e56f8dc6292773603d0d6aabbdd62a11ef721d1542d8
    # 873ac9ffea4dd04fa719e8920cd6938f0c23cd678af330939cff53c3d2855f34
    # b03ddf3ca2e714a6548e7495e2a03f5e824eaac9837cd7f159c67b90fb4b7342